In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, hamming_loss, jaccard_score
from sklearn.metrics import accuracy_score



/Users/andrewlutsky/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
embeddings = np.load("chest-xray14_embeddings.npy")
annotation = pd.read_csv('Data_Entry_2017.csv')
print(annotation)

             Image Index          Finding Labels  Follow-up #  Patient ID  \
0       00000001_000.png            Cardiomegaly            0           1   
1       00000001_001.png  Cardiomegaly|Emphysema            1           1   
2       00000001_002.png   Cardiomegaly|Effusion            2           1   
3       00000002_000.png              No Finding            0           2   
4       00000003_000.png                  Hernia            0           3   
...                  ...                     ...          ...         ...   
112115  00030801_001.png          Mass|Pneumonia            1       30801   
112116  00030802_000.png              No Finding            0       30802   
112117  00030803_000.png              No Finding            0       30803   
112118  00030804_000.png              No Finding            0       30804   
112119  00030805_000.png              No Finding            0       30805   

        Patient Age Patient Gender View Position  OriginalImage[Width  \
0 

In [25]:
# Create classification labels.
classifications = annotation["Finding Labels"]
classifications = [label.split("|") for label in classifications]
set_class = set()
for i in classifications:
    for j in i:
        set_class.add(j)

class_index = range(0, len(set_class))

map_class_index_label = dict(zip(set_class, class_index))



{'Emphysema': 0, 'Hernia': 1, 'No Finding': 2, 'Pleural_Thickening': 3, 'Effusion': 4, 'Nodule': 5, 'Mass': 6, 'Atelectasis': 7, 'Fibrosis': 8, 'Pneumonia': 9, 'Cardiomegaly': 10, 'Consolidation': 11, 'Pneumothorax': 12, 'Edema': 13, 'Infiltration': 14}


In [27]:
# Map labels to embeddings.
with open("selected_png_list.txt", "r") as f:
    lines = f.readlines()
    f.close()

# Read in image id.
lines = [line.split("/")[-1].strip("\n") for line in lines]
# Map image id to embeddings.
df_embeddings = pd.DataFrame()
df_embeddings["Image Index"] = lines
df_embeddings["embedding index"] = range(0, len(embeddings))
df_embeddings = pd.merge(df_embeddings, annotation, on = ["Image Index"], how="left" )
# Delete lines.
del lines

# Remove extraneous columns
df_embeddings = df_embeddings[["Image Index", "embedding index", "Finding Labels"]]

# Add features.
for i in range(len(embeddings[0])):
    df_embeddings[f"Feature {i}"] = embeddings[:, i]

# Add classifications.
for i in range(len(set_class)):
    df_embeddings[f"Label {i}"] = [0] * len(embeddings)
classifications = [classifications.split("|") for classifications in df_embeddings["Finding Labels"]]
labels = np.zeros((len(classifications), len(set_class)))

for i, lab in enumerate(classifications):
    for phenotype in lab:
        phen_index = map_class_index_label[phenotype]
        labels[i, phen_index] = 1

for i, lab in set_class:
    phen_index = map_class_index_label[lab]
    df_embeddings
print(labels)

print(df_embeddings)

[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
           Image Index  embedding index  \
0     00004118_002.png                0   
1     00005241_000.png                1   
2     00005681_020.png                2   
3     00006469_000.png                3   
4     00005921_000.png                4   
...                ...              ...   
9595  00000278_010.png             9595   
9596  00000274_000.png             9596   
9597  00000177_003.png             9597   
9598  00000376_000.png             9598   
9599  00000358_000.png             9599   

                                         Finding Labels  Feature 0  Feature 1  \
0     Edema|Effusion|Infiltration|Pleural_Thickening...  -0.053858   0.009951   
1                                            No Finding  -0.052940  -0.057987   
2     Cardiomegaly|Consolidation|Emphysema|Pneumothorax  -0.013555  -0.012684   
3  

/var/folders/66/9mt07h917lj07j24jtw0tqgw0000gn/T/ipykernel_21526/715179417.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_embeddings[f"Feature {i}"] = embeddings[:, i]
/var/folders/66/9mt07h917lj07j24jtw0tqgw0000gn/T/ipykernel_21526/715179417.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_embeddings[f"Feature {i}"] = embeddings[:, i]
/var/folders/66/9mt07h917lj07j24jtw0tqgw0000gn/T/ipykernel_21526/715179417.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

In [29]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='micro')  # 'micro' to aggregate the contributions of all classes
hamming = hamming_loss(y_test, y_pred)
jaccard = jaccard_score(y_test, y_pred, average='micro')

print(f"F1 Score: {f1}")
print(f"Hamming Loss: {hamming}")
print(f"Jaccard Score: {jaccard}")

F1 Score: 0.45903233649404324
Hamming Loss: 0.07725694444444445
Jaccard Score: 0.2978857683811928
